# Motion Planning, Assignment 1: Bug Algorithms

In this assignment, you will implement the Bug1 algorithm and test it in simulation. The assignment is divided into several excercises (with increasing level of difficulty) which are supposed to guide you towards the final implementation. If you haven't used Python before, check out the Python information on the course's Moodle page. 

The robot in this simulation is a point-shaped robot equipped with a laser scanner (cf. [Fig. 1](#fig_1)). Its position is given by $(x, y, \theta)$. The entities $x, y,$ and  $\theta$ are given in **global** coordinates.

As can be seen in [Fig. 1](#fig_1), the sensor readings are given with respect to an angle $\varphi$ in **local** coordinates. This means, that $\varphi$ is given **relative** to the robots orientation $\theta$.

<a id="fig_1"></a>
![Fig. 1: Robot scenario](https://drive.google.com/uc?export=view&id=11Ai2mL8tMdjGK6Sz7pXx-5d0pCXJ_jQO)

First, load upload necessary library files to Colab:

In [ ]:
# Upload data - Colab ONLY!
import io
import shutil

import requests

url = "https://raw.githubusercontent.com/ChiefGokhlayeh/mp/main/bugsim.py"

response = requests.get(url, allow_redirects=True)

with open("bugsim.py", "wb") as fin:
    shutil.copyfileobj(io.BytesIO(response.content), fin)

Now, load the necessary libraries:

In [ ]:
import math

import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML
from matplotlib import animation, rc

from bugsim import BugSim, normalizeAngle

## Exercise 1.1: Introduction

To get a first impression of the robot simulator, check out how a sample scenario is created. 

The following code section defines obstacles as set of axis-aligned rectangles. These ``objects`` are given as a list of points ``[xmin, ymin, xmax, ymax]`` where (xmin, ymin) denotes the lower left and (xmax, ymax) the upper right corner of the rectangle. You may select three different scenarios (and later create your own, of course).

Then, a simulator is created. The class ``BugSim`` will be used for simulation in this project. It can be initialized using 

````
simulator = BugSim(objects, goal=goal_point, 
                   sensor_resolution=20/180*np.pi, view_range=50,
                   safety_distance=5)
````

with arguments
* **objects:** the obstacles as a list of rectangles, see above.
* **goal:** the target position, e.g. (5,6)
* **view_range:** the maximum viewing distance of the sensor. Everything above that is indicated as np.inf.
* **sensor_resolution:** the angular resolution (in rad) of the sensor.
* **safety_distance:** the robot should keep that distance to obstacles. 


The BugSim class has several functions for moving the robot and accessing sensor data which will be described later. Here, the robot is spawned, and the sample scenario is drawn. 

In [ ]:
scenario = 1

if scenario == 1:
    # [[xmin, ymin, xmax, ymax], ...]
    objects = [
        [-5, -5, 205, 0],
        [-5, 100, 205, 105],
        [-5, -5, 0, 105],
        [200, -5, 205, 105],
        [70, 60, 150, 65],
        [70, 40, 75, 60],
    ]
    goal_point = (125, 80)
    spawn_pose = (100, 35, 90 / 180 * np.pi)

elif scenario == 2:
    objects = [
        [-5, -5, 205, 0],
        [-5, 200, 205, 205],
        [-5, -5, 0, 205],
        [200, -5, 205, 205],
        [70, 130, 150, 135],
        [70, 60, 150, 65],
        [70, 40, 75, 60],
    ]
    goal_point = (125, 180)
    spawn_pose = (100, 35, 90 / 180 * np.pi)

elif scenario == 3:
    objects = [
        [-5, -5, 205, 0],
        [-5, 200, 205, 205],
        [-5, -5, 0, 205],
        [200, -5, 205, 205],
        [70, 130, 150, 135],
        [70, 60, 150, 65],
        [70, 60, 75, 135],
        [150, 60, 155, 135],
    ]
    goal_point = (125, 100)
    spawn_pose = (100, 35, 90 / 180 * np.pi)

else:
    print("Error! Unknown scenario.")


# Create the simulator
simulator = BugSim(
    objects,
    goal=goal_point,
    sensor_resolution=20 / 180 * np.pi,
    view_range=50,
    safety_distance=5,
)

# Spawn the robot
simulator.spawn(x=spawn_pose[0], y=spawn_pose[1], theta=spawn_pose[2])

# Show
plt.figure()
simulator.drawBoard()
simulator.showRobot()
plt.title("obstacles");

**The BugSim class**

The BugSim class is a simple simulator for a point shaped robot equipped with a laser scanner.

The BugSim class has the following important attributes:
* **x, y, theta**: the 2d position and orientation (in rad) of the robot
* **view_range**: the maximum viewing distance of the sensor.
* **sensor_resolution**: the angular resolution (in rad) of the sensor.
* **safety_distance**: the robot should keep that distance to obstacles. 
* **goal**: the target position on the board
* **history**: all previously visited poses of the robot, general form: [[x0, y0, theta0], [x1, y1, theta1], ... ]

Its most important methods are:
* `spawn(x, y, theta)`: Spawn the robot at initial pose (x, y, theta).
* `getPose()`: Gets the current position and orientation of the robot. Returns: A list of values (x, y, theta) where theta is in rad.
* `forward(dist)`: Move robot about dist in forward direction. Please note that the robot will not move after a collision has been detected.
* `turn(dtheta)`: Turn robot about dtheta (rad) in mathematically positive direction
* `getNumMoves()`: Get number of motion steps the robot has done since it has been spawned.
* `getScan()`: Returns the sensor readings as numpy array of Nx2-shape where N is the number of measurements and
    * [:, 0] (first column) returns the relative angle phi (in rad)
    * [:, 1] (second column) returns the distance to the closes obstacle
* `getDistAtPhi(sensor_readings, phi)`: Gives the closest distance measurement to relative direction phi (rad). sensor_readings contains the sensor data.
* `checkCollision()`: Returns True iff the robot has collided with an obstacle.
* `drawBoard()`: Draws the board with obstacles and target position.
* `showRobot()`: Plot the robot at it's current location.

Let's begin with your first programming task: Using the sensor readings, draw all lidar scan rays from the robot to the obstacles. If the sensors cannot find an object in a specific direction, use the specifed view range as length for the ray. Your plot should resemble [Fig. 1](#fig_1) above.

In a second figure, plot the observed distance (y-axis) with respect to the view angle phi (x-axis).

**Hints:**
1. Use Matplotlib's ``plt.plot()`` for drawing.
2. The number of sensor scans can be obtained with `sensor_readings.shape[0]`

In [ ]:
plt.figure()
simulator.drawBoard()
simulator.showRobot()
if simulator.checkCollision():
    print("Collision occured.")
else:
    print("No collision.")

sensor_readings = simulator.getScan()
pose_x, pose_y, pose_theta = simulator.getPose()

xy = np.insert(
    np.vstack(
        [
            np.nan_to_num(sensor_readings[:, 1], posinf=simulator.view_range)
            * np.cos(sensor_readings[:, 0] + pose_theta)
            + pose_x,
            np.nan_to_num(sensor_readings[:, 1], posinf=simulator.view_range)
            * np.sin(sensor_readings[:, 0] + pose_theta)
            + pose_y,
        ]
    ).T,
    obj=range(sensor_readings.shape[0]),
    values=np.array([pose_x, pose_y]),
    axis=0,
)

simulator.showRobot()
plt.xlabel("x")
plt.ylabel("y")

plt.plot(xy[:, 0], xy[:, 1])

plt.figure()
plt.plot(
    sensor_readings[:, 0] * (180 / np.pi),
    np.nan_to_num(sensor_readings[:, 1], posinf=simulator.view_range),
)
plt.grid(True)
plt.xlabel("angle (°)")
plt.ylabel("distance (m)")

## Exercise 1.2: Moving the robot

The following is not a real exercise, but just an example how to move the robot in simulation. Motion commands are generated at random.

In [ ]:
plt.figure()
simulator.drawBoard()
simulator.spawn(x=spawn_pose[0], y=spawn_pose[1], theta=spawn_pose[2])
simulator.showRobot()

for i in range(100):
    simulator.turn(25.0 / 180.0 * np.pi * np.random.randn())
    simulator.forward(2.0 * np.random.rand())
    plt.plot(simulator.x, simulator.y, "bo")
    if simulator.checkCollision():
        break

## Excercise 1.3: Move towards goal

As a preparation for the Bug1 algorithm, write code that drives the robot from the current position towards the goal. The robot should stop once it reaches the goal, an obstacle (within the specified safety distance ``simulator.safety_distance``), or after 1000 movements.

The function ``getTowardsGoalState`` given below may be helpful for this task. 

When moving forward, use a ``stepSize=1``.

In [ ]:
def getTowardsGoalState(simulator):
    """Get status of robot when moving towards the goal.

    Args:
        simulator: the simulator class instance

    Returns:
        A tuple (goal_dist, toward_goal_phi, toward_goal_free_dist) where
        * goal_dist: distance to goal
        * toward_goal_phi: relative direction towards goal (rad)
        * toward_goal_free_dist: distance to next obstacle in the direction
            towards the goal (toward_goal_phi)
    """
    gx, gy = simulator.goal
    x, y, theta = simulator.getPose()

    goal_dist = np.sqrt((gx - x) ** 2 + (gy - y) ** 2)

    sensor_readings = simulator.getScan()

    toward_goal_theta = np.arctan2(gy - y, gx - x)
    toward_goal_phi = normalizeAngle(toward_goal_theta - theta)
    toward_goal_free_dist = simulator.getDistAtPhi(sensor_readings, toward_goal_phi)

    return goal_dist, toward_goal_phi, toward_goal_free_dist


def getTowardsObstacleState(simulator):
    """Get status of robot with respect to nearest obstacle.

    Args:
        simulator: the simulator class instance

    Returns:
        A tuple (towards_obstacle_phi, towards_obstacle_dist) where
        * towards_obstacle_phi: relative direction towards nearest obstacle (rad)
        * towards_obstacle_dist: distance to nearest obstacle
    """
    sensor_readings = simulator.getScan()

    towards_obstacle_idx = np.argmin(sensor_readings[:, 1])
    towards_obstacle_phi = sensor_readings[towards_obstacle_idx, 0]
    towards_obstacle_dist = sensor_readings[towards_obstacle_idx, 1]

    return towards_obstacle_phi, towards_obstacle_dist

In [ ]:
stepSize = 1.0
simulator = BugSim(
    objects,
    goal=goal_point,
    sensor_resolution=5 / 180 * np.pi,
    view_range=50,
    safety_distance=5,
)
simulator.spawn(x=spawn_pose[0], y=spawn_pose[1], theta=spawn_pose[2])

plt.figure()
simulator.drawBoard()
simulator.showRobot()

goal_dist, toward_goal_phi, toward_goal_free_dist = getTowardsGoalState(simulator)
toward_obstacle_phi, toward_obstacle_dist = getTowardsObstacleState(simulator)

while (
    simulator.getNumMoves() < 100
    and goal_dist >= stepSize
    and not simulator.checkCollision()
):
    simulator.turn(toward_goal_phi)
    simulator.forward(stepSize)

    plt.plot(simulator.x, simulator.y, "bo")

    goal_dist, toward_goal_phi, toward_goal_free_dist = getTowardsGoalState(simulator)
    toward_obstacle_phi, toward_obstacle_dist = getTowardsObstacleState(simulator)

simulator.showRobot()

## Excercise 1.4: Move around obstacle

The second part of preparing Bug1 is circumventing an obstacle at a safety distance specified in ``simulator.safety_distance``. Do so by implementing the closed-loop following as described in the lecture and have the robot do 400 steps. Don't worry about finding a leave point for now.

The robot is initially placed on the border of the obstacle. Please use the same step size as above and make sure that your are using ``scenario=1`` when testing this. Write code both for turning left and right at the obstacle. 

Again, the functions ``getTowardsGoalState`` and ``getTowardsObstacleState`` may be helpful.

In [ ]:
assert scenario == 1, "Please use scenario 1 for this excercise!"

simulator.spawn(x=100, y=55, theta=90 / 180.0 * np.pi)

plt.figure()
simulator.drawBoard()
simulator.showRobot()

go_right = False

goal_dist, toward_goal_phi, toward_goal_free_dist = getTowardsGoalState(simulator)
toward_obstacle_phi, toward_obstacle_dist = getTowardsObstacleState(simulator)

while (
    simulator.getNumMoves() < 400
    and goal_dist >= stepSize
    and not simulator.checkCollision()
):
    if toward_obstacle_dist < simulator.safety_distance:
        simulator.turn(toward_obstacle_phi + np.pi / 2)
    else:
        simulator.turn(toward_goal_phi)
    simulator.forward(stepSize)

    plt.plot(simulator.x, simulator.y, "bo")

    goal_dist, toward_goal_phi, toward_goal_free_dist = getTowardsGoalState(simulator)
    toward_obstacle_phi, toward_obstacle_dist = getTowardsObstacleState(simulator)

simulator.showRobot()

## Excercise 1.5: Putting everything together

Now, write the full Bug1 algorithm as described in the lecture slides.

**Hints**: 
1. First, implement the two functions ``moveTowardsGoal`` and ``moveAroundObstacle``  based on the previous excercises.  
2. You may select a fixed direction (left or right) when returning to the best leave point. Choosing the shortest distance to the leave point may be omitted for simplicity.
3. When checking if a leave point or safety distance to obstacle is reached, keep in mind that the robot may not hit leave points or safety distances to obstacles perfectly accurate.

In [ ]:
stepSize = 1.0
maxMoves = 3000

simulator = BugSim(
    objects,
    goal=goal_point,
    sensor_resolution=5 / 180 * np.pi,
    view_range=50,
    safety_distance=5,
)
simulator.spawn(x=spawn_pose[0], y=spawn_pose[1], theta=spawn_pose[2])

plt.figure()
simulator.drawBoard()
simulator.showRobot()


def moveTowardsGoal(simulator, stepSize, maxTotalMoves=1000):
    """Move in a straigt line towards the goal.

    This will move the robot towards the obstacle until the goal is reached,
    an obstacle is encountered within the safety distance, or the maximum
    number of steps has been exceeded.

    Args:
        simulator: the simulator object
        stepSize: the distance to move in each forward step
        maxTotalMoves: stop execution after maxTotalMoves

    Returns:
        True if the goal has been reached, False otherwise.
    """

    while True:
        goal_dist, toward_goal_phi, _ = getTowardsGoalState(simulator)
        _, toward_obstacle_dist = getTowardsObstacleState(simulator)

        if (
            goal_dist <= stepSize
            or toward_obstacle_dist <= simulator.safety_distance
            or simulator.getNumMoves() >= maxTotalMoves
            or simulator.checkCollision()
        ):
            break

        simulator.turn(toward_goal_phi)
        simulator.forward(stepSize)

        plt.plot(simulator.x, simulator.y, "bo")

    return goal_dist <= stepSize


def moveAroundObstacle(
    simulator, stepSize, targetPos, go_right=False, maxTotalMoves=1000
):
    """Circumvent an obstacle.

    This will move the robot towards the obstacle until the goal
    is reached, the robot arrives close to a given target position,
    or the maximum number of steps has been exceeded.

    Args:
        simulator: the simulator object
        stepSize: the distance to move in each forward step
        targetPos: the target position at the border of the obstacle
            (e.g. the hit point or the leave point)
        go_right: True if robot should go right at obstacle
        maxTotalMoves: stop execution after maxTotalMoves

    Returns:
        True if the goal has been reached, False otherwise.
    """

    position_history = targetPos.reshape(1, 2)

    while True:
        goal_dist, _, _ = getTowardsGoalState(simulator)
        toward_obstacle_phi, toward_obstacle_dist = getTowardsObstacleState(simulator)

        x, y, _ = simulator.getPose()

        if (
            goal_dist <= stepSize
            or (
                np.linalg.norm(np.array([x, y]) - targetPos, ord=2) <= stepSize
                and position_history.shape[0] > 1
            )
            or simulator.getNumMoves() >= maxTotalMoves
            or simulator.checkCollision()
        ):
            break

        simulator.turn(
            toward_obstacle_phi + (-(np.pi / 2) if go_right else +(np.pi / 2))
        )
        simulator.forward(stepSize)

        plt.plot(simulator.x, simulator.y, "bo")

        x, y, _ = simulator.getPose()
        position_history = np.vstack([position_history, np.array([x, y])])

    closest_idx = np.argmin(
        np.linalg.norm(position_history - simulator.goal, ord=2, axis=1)
    )
    closest = position_history[closest_idx, :]

    plt.plot(closest[0], closest[1], "co")

    if position_history.shape[0] // 2 <= closest_idx:
        go_right = not go_right

    targetPos = closest

    while True:
        goal_dist, _, _ = getTowardsGoalState(simulator)
        toward_obstacle_phi, toward_obstacle_dist = getTowardsObstacleState(simulator)

        x, y, _ = simulator.getPose()

        if (
            goal_dist <= stepSize
            or (
                np.linalg.norm(np.array([x, y]) - targetPos, ord=2) <= stepSize
                and position_history.shape[0] > 1
            )
            or simulator.getNumMoves() >= maxTotalMoves
            or simulator.checkCollision()
        ):
            break

        simulator.turn(
            toward_obstacle_phi + (-(np.pi / 2) if go_right else +(np.pi / 2))
        )
        simulator.forward(stepSize)

        plt.plot(simulator.x, simulator.y, "mo")

    return goal_dist <= stepSize


while True:
    if not moveTowardsGoal(simulator, stepSize):
        x, y, _ = simulator.getPose()
        moveAroundObstacle(simulator, stepSize, np.array([x, y]), go_right=True)
    else:
        break

simulator.showRobot()

If you were able to solve the task, you may use the next section to create an anmiation of your robot:

In [ ]:
anim = simulator.animate(skipFrames=2)
rc("animation", html="jshtml")
anim

## Optional excercises

* Create your own scenarios and test the algorithm.
* Write code for Bug2 or Tangent Bug.